### Challenge 9

In [6]:
def pad_pkcs7(message: bytes, block_size: int) -> bytes:
    pad_len = block_size - len(message) % block_size
    return message + b'\x04' * pad_len
    
assert pad_pkcs7('YELLOW SUBMARINE'.encode(), 20).decode() == "YELLOW SUBMARINE\x04\x04\x04\x04"

### Challenge 10

In [31]:
from Crypto.Cipher import AES

def ensure_bytes(inp):
    return inp.encode() if type(inp) == str else inp

def xor(a, b):
    assert len(a) == len(b)
    out = b''
    for a_bit, b_bit in zip(a,b):
        out += bytes([a_bit ^ b_bit])
    return out

xor(b'\x00\x01', b'\x01\x01')

b'\x01\x00'

In [38]:
def aes_cbc_encrpyt(iv, key, ptxt):
    iv = ensure_bytes(iv)
    key = ensure_bytes(key)
    ptxt = ensure_bytes(ptxt)

    padded_ptxt = pad_pkcs7(ptxt, len(key))
    cipher = AES.new(key, AES.MODE_ECB)
    
    out = [iv]
    for st in range(0,len(padded_ptxt),len(key)):
        blk = padded_ptxt[st:st+len(key)]
        out.append(cipher.encrypt(xor(out[-1], blk)))
    ctxt = out[1:]
    return b''.join(ctxt)

def aes_cbc_decrpyt(iv, key, ctxt):
    iv = ensure_bytes(iv)
    key = ensure_bytes(key)
    
    cipher = AES.new(key, AES.MODE_ECB)
    
    out = [iv]
    for st in range(0, len(ctxt), len(key)):
        blk = ctxt[st: st+len(key)]
        out.append(xor(cipher.decrypt(blk), out[-1]))
    ptxt = out[1:]
    return b''.join(ptxt)

In [44]:
key = b'YELLOW SUBMARINE'
iv = b'0'*len(key)
msg = b'this is a test message'[:16]
ctxt = aes_cbc_encrpyt(iv, key, msg)
ctxt

b'1gZ\xc4\xc0zr\xa4\x04\xc6\xa9W\x80\xa5\xd6\x9e7}\xc0\x14=&\x1b\xafz\x0b\xedn\xfd\x0b\x846'

In [45]:
aes_cbc_decrpyt(iv, key, ctxt)

b'this is a test mA\x0b7\xb3\xe4\x17\x05\x80a\xe2\xd96\xf7\xd5\xf2\xf7'